# 1. Import libraries

In [1]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 6.8 MB/s 
     |████████████████████████████████| 86 kB 7.6 MB/s 
     |████████████████████████████████| 448 kB 64.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
!pip install gensim

In [4]:
#colab에서 실행 중이라면...
!git clone https://github.com/hukim1112/comment_classifier.git
import os
os.chdir('/content/comment_classifier')

Cloning into 'comment_classifier'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 74 (delta 0), reused 1 (delta 0), pack-reused 71
Unpacking objects: 100% (74/74), done.


In [5]:
import tensorflow as tf
from konlpy.tag import Twitter
from collections import Counter
import pandas as pd
import numpy as np
import gensim
from matplotlib import pyplot as plt

keras = tf.keras
t = Twitter()

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [6]:
t.morphs("나는 배고프다")

['나', '는', '배고프다']

# 2. fit tokenizer to our datasets

In [7]:
from vectorizer import BaseVectorizer
tokenizer = BaseVectorizer(t.morphs)

In [8]:
df = pd.read_csv('train_intent.csv')

In [11]:
len(df)

3591

In [12]:
tokenizer.fit(df['question'].values)

scanning was done                                        
1405 terms are recognized


In [13]:
tokenizer.vocabulary_

{'_PAD_': 0,
 '_UNK_': 1,
 '_STA_': 2,
 '_EOS_': 3,
 '오늘': 4,
 '알려줘': 5,
 '명언': 6,
 '이': 7,
 '누구': 8,
 '알려줘요': 9,
 '날씨': 10,
 '주': 11,
 '지금': 12,
 '해줘': 13,
 '시간': 14,
 '뭐': 15,
 '노래': 16,
 '추천': 17,
 '상태': 18,
 '들려줘': 19,
 '몇': 20,
 '말': 21,
 '공기': 22,
 '이슈': 23,
 '요': 24,
 '무슨': 25,
 '좀': 26,
 '이번': 27,
 '맛집': 28,
 '요즘': 29,
 '나': 30,
 '알려줄래': 31,
 '주변': 32,
 '내일': 33,
 '해줘요': 34,
 '알려주라': 35,
 '지': 36,
 '며칠': 37,
 '알려줄래요': 38,
 '의': 39,
 '좋은': 40,
 '뉴스': 41,
 '사건': 42,
 '현재': 43,
 '이제': 44,
 '곧': 45,
 '번역': 46,
 '날짜': 47,
 '에요': 48,
 '에게': 49,
 '되는': 50,
 '인가요': 51,
 '뭔': 52,
 '거': 53,
 '핫': 54,
 '미세먼지': 55,
 '하나': 56,
 '시야': 57,
 '니': 58,
 '가장': 59,
 '화제': 60,
 '어디': 61,
 '중국어': 62,
 '있나요': 63,
 '이야': 64,
 '요일': 65,
 '인지': 66,
 '힘': 67,
 '들려줘요': 68,
 '일': 69,
 '에': 70,
 '마스크': 71,
 '어떻게': 72,
 '우산': 73,
 '해주라': 74,
 '가': 75,
 '해주세요': 76,
 '멋진': 77,
 '결과': 78,
 '바람': 79,
 '프랑스어': 80,
 '다음': 81,
 '있니': 82,
 '전': 83,
 '누군지': 84,
 '알려주세요': 85,
 '라': 86,
 '영어': 87,
 '음악': 88,
 '시': 89,


# 3. data preprocessing

In [14]:
label_to_id = {t:i for i,t in enumerate(df.intent.unique())}
id_to_label = {i:t for i,t in enumerate(df.intent.unique())}

In [15]:
print(label_to_id)

{'음악': 0, '번역': 1, '뉴스': 2, '명언': 3, '달력': 4, '먼지': 5, '인물': 6, '시간': 7, '이슈': 8, '날씨': 9, '맛집': 10}


In [ ]:
# df.intent = df.intent.map(lambda x : label_index[x])
# print(df.head(10))

In [16]:
MAX_LENGTH = 40
def tokenize_and_filter(sentences, labels):
  inputs, outputs = [], []
  
  for sentence, label in zip(sentences, labels):
    # tokenize sentence
    tokenized_sentence = tokenizer.encode_a_doc_to_list(sentence)
    # check tokenized sentence max length
    if len(tokenized_sentence) <= MAX_LENGTH:
      inputs.append(tokenized_sentence)
      outputs.append(label_to_id[label])
  
  # pad tokenized sentences
  padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      inputs, maxlen=MAX_LENGTH, padding='post', 
      value = tokenizer.vocabulary_['_PAD_']) # value = 0
  
  return padded_inputs, outputs

In [17]:
inputs, outputs = tokenize_and_filter(df.question, df.intent)

In [18]:
print('encoded input : ', inputs[0], 'label : ', outputs[0], 'original input sentence : ', tokenizer.decode_from_list(inputs[0]))

encoded input :  [185  73  19   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0] label :  0 original input sentence :  ['에픽하이', '우산', '들려줘', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_']


In [45]:
BATCH_SIZE = 16
BUFFER_SIZE = 7836

# decoder inputs use the previous target as input
# remove START_TOKEN from targets
dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))
dataset = dataset.shuffle(3000, reshuffle_each_iteration=False)
val_ds = dataset.take(500)
train_ds = dataset.skip(500)

def configure_ds(dataset):
  dataset = dataset.cache()
  dataset = dataset.shuffle(BUFFER_SIZE)
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
  return dataset
train_ds = configure_ds(train_ds)
val_ds = configure_ds(val_ds)

In [46]:
for x, y in train_ds.take(1):
    print(x, y)
    print('-----------------------------------------------')
    print(x.shape, y.shape)

tf.Tensor(
[[  90   87  146   21   13    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [  42   25   19    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [  27   11   10   97    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [1181    7    8    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [  60   39   23    9    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 

# 4. model design

In [47]:
print(len(label_to_id.values()))

11


In [48]:
def get_model():
    return tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.n_vocabs, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(label_to_id.values()), activation='softmax')
])

In [49]:
model = get_model()

In [50]:
LEARNING_RATE = 0.0001

In [51]:
model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [52]:
for i, j in train_ds.take(1):
  print(i.shape, j)

(16, 40) tf.Tensor([ 0  7  9  4  3  8  6  9  7 10  4  8  3  5  6  0], shape=(16,), dtype=int32)


In [53]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
194/194 [==============================] - 7s 14ms/step - loss: 2.3829 - sparse_categorical_accuracy: 0.1495 - val_loss: 2.3736 - val_sparse_categorical_accuracy: 0.1780
Epoch 2/10
194/194 [==============================] - 2s 9ms/step - loss: 2.3048 - sparse_categorical_accuracy: 0.2863 - val_loss: 2.1883 - val_sparse_categorical_accuracy: 0.4400
Epoch 3/10
194/194 [==============================] - 2s 9ms/step - loss: 1.8855 - sparse_categorical_accuracy: 0.5205 - val_loss: 1.6386 - val_sparse_categorical_accuracy: 0.4800
Epoch 4/10
194/194 [==============================] - 2s 9ms/step - loss: 1.3501 - sparse_categorical_accuracy: 0.7189 - val_loss: 1.1783 - val_sparse_categorical_accuracy: 0.7800
Epoch 5/10
194/194 [==============================] - 2s 9ms/step - loss: 0.9515 - sparse_categorical_accuracy: 0.8447 - val_loss: 0.9054 - val_sparse_categorical_accuracy: 0.8700
Epoch 6/10
194/194 [==============================] - 2s 9ms/step - loss: 0.6896 - sparse_categoric

In [54]:
def question_processing(sentences):
    inputs = []
    for sentence in sentences:
        # tokenize sentence
        tokenized_sentence = tokenizer.encode_a_doc_to_list(sentence)
        # check tokenized sentence max length
        if len(tokenized_sentence) <= MAX_LENGTH:
            inputs.append(tokenized_sentence)
        else:
            print('입력이 너무 길어요.')
    # pad tokenized sentences
    padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    inputs, maxlen=MAX_LENGTH, padding='post', 
    value = tokenizer.vocabulary_['_PAD_']) # value = 0
    return padded_inputs

In [55]:
question_processing(["나는 전주 날씨 궁금함"])

array([[ 30, 130, 135,  10,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0]], dtype=int32)

In [56]:
input_sentence = question_processing(['서울 날씨 어때?', 
                                      '나는 전주 날씨 궁금함',
                                      '안중근 의사는 누구야?',
                                      '이순신 장군은 어떤 분이니?',
                                      '명동 맛있는 음식점 있니?'
                                     ])

In [57]:
model.predict(input_sentence)

array([[3.34740989e-02, 6.15719819e-06, 1.22649437e-02, 2.12770331e-07,
        1.89632282e-03, 4.59708885e-04, 8.76961130e-05, 1.77765619e-02,
        2.87766289e-03, 9.16435838e-01, 1.47208460e-02],
       [6.98929280e-02, 5.27820202e-06, 1.07014002e-02, 1.43738885e-06,
        8.29131901e-03, 6.28543377e-04, 1.49984407e-04, 4.77604792e-02,
        9.08961147e-03, 8.38718295e-01, 1.47606730e-02],
       [1.49378367e-02, 2.03916095e-02, 9.69733286e-04, 4.43652049e-02,
        5.52695524e-03, 1.85251562e-03, 8.79908085e-01, 9.75205476e-05,
        2.23294563e-07, 7.57845934e-04, 3.11925020e-02],
       [1.50376901e-01, 2.01502335e-05, 5.75422775e-03, 5.94119156e-05,
        1.06199585e-01, 1.39980786e-03, 1.38088898e-03, 5.30403256e-01,
        1.01618953e-02, 1.59425795e-01, 3.48181315e-02],
       [1.31182894e-01, 3.84239502e-05, 1.79505395e-03, 1.37859810e-04,
        2.35977233e-03, 1.39308177e-04, 1.52279474e-02, 3.27018239e-02,
        2.09725622e-05, 3.30121890e-02, 7.83383727e-

In [58]:
prediction = np.argmax(model.predict(input_sentence), axis=1)
print(prediction)

[ 9  9  6  7 10]


In [59]:
for p in prediction:
    print(id_to_label[p])

날씨
날씨
인물
시간
맛집


In [ ]:
del model

# 데이터 추가해보기

In [ ]:
names = ['안중근', '이순신', '세종대왕', '김광석', '아이유', '에미넴', '이건희', '고아라', '유재석', '한석희', '최민성']
def question_generator(names):
    question = []
    for name in names:
        s1 = name+'는 어떤 분이야?'
        s2 = name+'은 어떤 사람이니?'
        s3 = name+'이란 사람에 대해 궁금해'
        question = question+[s1, s2, s3]
    return question
question = question_generator(names)

In [ ]:
question

In [ ]:
new_data = {'question' : question, 'intent' : ['인물']*len(question)}
add_df = pd.DataFrame(new_data, columns=('question', 'intent'))

In [ ]:
add_df.head(5)

In [ ]:
print(len(df), len(add_df))

In [ ]:
new_df = pd.concat([df, add_df])
print(len(new_df))

In [ ]:
tokenizer.fit(new_df['question'].values)

In [ ]:
new_inputs, new_outputs = tokenize_and_filter(new_df.question, new_df.intent)

BATCH_SIZE = 16
BUFFER_SIZE = 7836

# decoder inputs use the previous target as input
# remove START_TOKEN from targets
dataset = tf.data.Dataset.from_tensor_slices((new_inputs, new_outputs))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
new_model = get_model()
LEARNING_RATE = 0.0001
new_model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
new_model.fit(dataset, epochs=10)

In [ ]:
input_sentence = question_processing(['서울 날씨 어때?', 
                                      '나는 전주 날씨 궁금함',
                                      '안중근 의사는 누구야?',
                                      '박소희는 어떤 사람인지 궁금해.',
                                      '명동 맛있는 음식점 있니?'
                                     ])

In [ ]:
new_model.predict(input_sentence)

In [ ]:
prediction = np.argmax(new_model.predict(input_sentence), axis=1)

In [ ]:
for p in prediction:
    print(id_to_label[p])